 # AI-Platform を利用した Titanic データ予測
 GCP の AI-Platform へモデルをデプロイし予測する。
 ※ 2020/02/12 時点での情報及び動作確認となります。

 ## GCP アカウント登録
 [【画像で説明】Google Cloud Platform (GCP)の無料トライアルでアカウント登録](https://qiita.com/komiya_____/items/14bd06d0866f182ae912)

 ## Google Cloud SDK のインストール
 [Google Cloud SDK のインストール ~ 初期化](https://qiita.com/komiya_____/items/5af0dcc8639fad9fee29)

 ## ログイン
 gcloud コマンドをつかって GCP をいじるのに必要になります。

In [1]:
! gcloud auth login


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?code_challenge=O31FrcmSd8GkraDIn9UX0kYm1O5EeXgtoFodrkU6wDA&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth



You are now logged in as [edsa9654@gmail.com].
Your current project is [mypj-id].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


 ## プロジェクト ID とプロジェクトネームを設定
 ID はカブり禁止です。

In [2]:
PROJECT_ID = 'anata-no-project-id' #@param {type: "string"}
PROJECT_NAME = 'anata-no-project-name' #@param {type: "string"}


In [3]:
! gcloud config list



[compute]
region = us-central1
zone = us-central1-a
[core]
account = edsa9654@gmail.com
disable_usage_reporting = True
project = mypj-id

Your active configuration is: [default]
